# NimbleSearchRetriever

`NimbleSearchRetriever` 使开发者能够构建 RAG 应用和 AI Agent，以搜索、访问和检索来自网络任何地方的在线信息。

`NimbleSearchRetriever` 利用 Nimble 的 Data APIs，以高效、可扩展且有效的方式执行搜索查询并检索网络数据。
它有两种模式：

*   **搜索与检索**：执行搜索查询，获取前 N 个结果的 URL，并从这些 URL 中检索文本。
*   **检索**：提供一个 URL 列表，并从这些 URL 中检索文本/数据。

如果您想了解更多关于底层 Nimble API 的信息，请访问[此处](https://docs.nimbleway.com/nimble-sdk/web-api/web-api-overview)的文档。

## 设置

要开始使用 `NimbleSearchRetriever`，您需要先在 Nimble 开设一个账户并订阅一个计划。Nimble 提供免费试用，[您可以在此处注册](https://app.nimbleway.com/signup?returnTo=/pipelines/nimbleapi)。

有关可用计划的更多信息，请参阅我们的[定价页面](https://www.nimbleway.com/pricing)。

注册后，您需要进入“Pipelines”页面中的“Nimble API”管道，这将为您提供 API 凭证（一个 base64 令牌），用于认证您的检索器：

![example-image](

In [1]:
import getpass
import os

os.environ["NIMBLE_API_KEY"] = getpass.getpass()

有关身份验证过程的更多信息，请参阅 [Nimble APIs Authentication Documentation](https://docs.nimbleway.com/nimble-sdk/web-api/nimble-web-api-quick-start-guide/nimble-apis-authentication)。

如果您想为单个查询获取自动化跟踪，可以通过取消注释下方代码来设置您的 [LangSmith](https://docs.smith.langchain.com/) API 密钥：

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### Installation

检索器是 langchain providers 的一部分，包含在 PyPI 上提供的 langchain-nimble 包中。

In [ ]:
%pip install -U langchain-nimble langchain-openai

## 实例化

现在我们可以实例化我们的检索器了：

In [3]:
from langchain_nimble import NimbleSearchRetriever

retriever = NimbleSearchRetriever(k=3)

## 用法

`NimbleSearchRetriever` 具有以下参数：

*   `k` (可选) 整数 - 要返回的结果数量（小于或等于 20）
*   `api_key` (可选) 字符串 - Nimble 的 API 密钥，可以在实例化检索器时直接发送，或通过环境变量 (`NIMBLE_API_KEY`) 发送
*   `search_engine` (可选) 字符串 - 查询将通过其执行的搜索引擎，您可以从以下选项中选择：
    * `google_search` (默认值) - 谷歌搜索引擎
    * `bing_search` - 必应搜索引擎
    * `yandex_search` - 俄罗斯 Yandex 搜索引擎
*   `render` (可选) 布尔值 - 启用或禁用目标页面的 Javascript 渲染（如果启用，结果返回可能会变慢）
*   `locale` (可选) 字符串 - 用于 URL 请求的 LCID 标准区域设置。或者，用户可以使用 `auto` 来根据国家/地区定位自动设置区域设置。
*   `country` (可选) 字符串 - 用于访问目标 URL 的国家/地区，使用 ISO Alpha-2 国家/地区代码，例如 US、DE、GB
*   `parsing_type` (可选) 字符串 - 返回的 `page_content` 的文本结构
    * `plain_text` (默认值) - 仅从 html 中提取文本
    * `markdown` - Markdown 格式
    * `simplified_html` - 原始 html 文档的压缩版本（约占原始 html 大小的 8%）
*   `links` (可选) 字符串数组 - 要抓取的网站链接数组，如果选择此项，将返回这些网站的原始 html 内容 **(这将激活第二种模式)**

您可以在 [Nimble 的文档](https://docs.nimbleway.com/nimble-sdk/web-api/vertical-endpoints/serp-api/real-time-search-request#request-options) 中阅读有关每个参数的更多信息。

### 使用搜索查询字符串的搜索与检索模式示例

**获取单个文档将产生以下结果：**

In [4]:
import json

query = "Latest trends in artificial intelligence"
example_doc = retriever.invoke(query)[0]
print("Page Content: \n", json.dumps(example_doc.page_content, indent=2))
print("Metadata: \n", json.dumps(example_doc.metadata, indent=2))

Page Content: 
 "8 AI and machine learning trends to watch in 2025 | TechTarget\nSearch Enterprise AI\nSearch the TechTarget Network\nLogin\nRegister\nExplore the Network\nTechTarget Network\nBusiness Analytics\nCIO\nData Management\nERP\nSearch Enterprise AI\nAI Business Strategies\nAI Careers\nAI Infrastructure\nAI Platforms\nAI Technologies\nMore Topics\nApplications of AI\nML Platforms\nOther Content\nNews\nFeatures\nTips\nWebinars\n2024 IT Salary Survey Results\nSponsored Sites\nMore\nAnswers\nConference Guides\nDefinitions\nOpinions\nPodcasts\nQuizzes\nTech Accelerators\nTutorials\nVideos\nFollow:\nHome\nAI business strategies\nTech Accelerator\nWhat is enterprise AI? A complete guide for businesses\nPrev\nNext\n8 jobs that AI can't replace and why\n10 top artificial intelligence certifications and courses for 2025\nDownload this guide1\nX\nFree Download\nA guide to artificial intelligence in the enterprise\nThis wide-ranging guide to artificial intelligence in the enterprise pro

**当调用相同的查询而不使用单个文档时，结果将是：**

In [5]:
retriever.invoke(query)

[Document(metadata={'title': '8 AI and machine learning trends to watch in 2025', 'snippet': 'Jan 3, 2025 — 1. Hype gives way to more pragmatic approaches · 2. Generative AI moves beyond chatbots · 3. AI agents are the next frontier · 4. Generative AI\xa0...', 'url': 'https://www.techtarget.com/searchenterpriseai/tip/9-top-AI-and-machine-learning-trends', 'position': 1, 'entity_type': 'OrganicResult'}, page_content='8 AI and machine learning trends to watch in 2025 | TechTarget\nSearch Enterprise AI\nSearch the TechTarget Network\nLogin\nRegister\nExplore the Network\nTechTarget Network\nBusiness Analytics\nCIO\nData Management\nERP\nSearch Enterprise AI\nAI Business Strategies\nAI Careers\nAI Infrastructure\nAI Platforms\nAI Technologies\nMore Topics\nApplications of AI\nML Platforms\nOther Content\nNews\nFeatures\nTips\nWebinars\n2024 IT Salary Survey Results\nSponsored Sites\nMore\nAnswers\nConference Guides\nDefinitions\nOpinions\nPodcasts\nQuizzes\nTech Accelerators\nTutorials\nVi

### 检索模式的 URL 数组示例

In [6]:
retrieverMode2 = NimbleSearchRetriever(links=["example.com"])
retrieverMode2.invoke(input="")

[Document(metadata={'title': None, 'snippet': None, 'url': 'https://example.com', 'position': None, 'entity_type': 'HtmlContent'}, page_content='<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700

## 在链中使用

与其他检索器一样，NimbleSearchRetriever 可以通过 [链](/docs/how_to/sequence/) 整合到 LLM 应用中。

我们需要一个 LLM 或聊天模型：

In [7]:
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = getpass.getpass()
llm = ChatOpenAI(model="gpt-4o", temperature=0)

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

Context: {context}

Question: {question}"""
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [9]:
chain.invoke("Who is the CEO of Nimbleway?")

'The CEO of Nimble Way is Uriel Knorovich.'